# Check Current File Management Setup
Review the existing code that handles file management, especially any code that uses google.colab or mounts Google Drive.

In [1]:
# Colab GPU runtime recommended (Runtime » Change runtime type • GPU) spip -q install ultralytics opencv-python-headless ffmpeg-python
from ultralytics import YOLO
from pathlib import Path
from IPython.display import display, Image
import cv2
import numpy as np
import shutil, glob, os, torch, random, csv
# data = open('DeepSea_ObjectDetection/Data') # save everything here

ModuleNotFoundError: No module named 'ultralytics'

In [28]:
# Base path to the 'Data' directory
base_path = Path('DeepSea Video Samples')

# Collect all .mp4 files from the base path
video_files = list(base_path.rglob('*.mp4'))

# Print results
print(f'Found {len(video_files)} .mp4 files:')
for video in video_files:
    print(video)

Found 12 .mp4 files:
DeepSea Video Samples/GH144320-008.mp4
DeepSea Video Samples/GH034320-003.mp4
DeepSea Video Samples/GH134320-012.mp4
DeepSea Video Samples/GH064320-002.mp4
DeepSea Video Samples/GH094320-011.mp4
DeepSea Video Samples/GH104320-001.mp4
DeepSea Video Samples/GH074320-009.mp4
DeepSea Video Samples/GH054320-007.mp4
DeepSea Video Samples/GH124320-005.mp4
DeepSea Video Samples/GH044320-006.mp4
DeepSea Video Samples/GH114320-004.mp4
DeepSea Video Samples/GH084320-010.mp4


In [29]:
for video_path in video_files:
    size = video_path.stat().st_size
    print(f"{video_path.name}: {size / 1_000_000:.2f} MB")

GH144320-008.mp4: 4003.45 MB
GH034320-003.mp4: 4003.56 MB
GH134320-012.mp4: 4003.44 MB
GH064320-002.mp4: 4003.55 MB
GH094320-011.mp4: 4003.69 MB
GH104320-001.mp4: 4003.33 MB
GH074320-009.mp4: 4003.50 MB
GH054320-007.mp4: 4003.52 MB
GH124320-005.mp4: 4003.61 MB
GH044320-006.mp4: 4003.47 MB
GH114320-004.mp4: 4003.68 MB
GH084320-010.mp4: 4003.54 MB


In [30]:
%pip install ultralytics

from ultralytics import YOLO

# Load a YOLOv8 model (you can use 'yolov8n.pt', 'yolov8s.pt', etc.)
model = YOLO('yolov8n.pt')  # or 'yolov8n.yaml' if training from scratch

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
output_dir = Path('dataset/images/train')
output_dir.mkdir(parents=True, exist_ok=True)

for video_path in video_files:
    cap = cv2.VideoCapture(str(video_path))
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_name = output_dir / f'{video_path.stem}_{frame_idx}.jpg'
        cv2.imwrite(str(frame_name), frame)
        frame_idx += 1
    cap.release()

In [34]:
model.train(data='data.yaml', epochs=50, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.143 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.141 🚀 Python-3.11.1 torch-2.7.0 CPU (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pa

RuntimeError: Dataset 'data.yaml' error ❌ 'data.yaml' does not exist

In [ ]:
# Load the best trained model
model = YOLO('runs/detect/train/weights/best.pt')

# Run inference
results = model('path_to_image_or_video.mp4')
results[0].show()  # visualize predictions

In [ ]:
shutil.copy(
    'runs/detect/train/weights/best.pt',
    'my_models/deep_sea_detector.pt'
)